<a href="https://colab.research.google.com/github/heesukjang/W209_DataViz_Summer2023/blob/main/Unit_Assignments/Week_09_Assignment_ChoroplethMap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## W209 Data Visualization, Summer 2023
**Week 9 Assignment<br>
Heesuk Jang**

### All tracks<br>

This weeks assignment is to plot a county-level choropleth using the "unemployment" dataset available here: https://cdn.jsdelivr.net/npm/vega-datasets@1.29.0/data/.

You can load this dataset in Observable by importing the JS package, see example here.

In Python, you can load this dataset directly from

**from vega_datasets import data<br>
source = data.unemployment.url**

**For Tableau**, you can download the TSV file from the link.

In [ ]:
!pip install nx_altair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import altair as alt
import pandas as pd
from vega_datasets import data

source = data.unemployment.url
source

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/unemployment.tsv'

In [6]:
counties = alt.topo_feature(data.us_10m.url, 'counties')

choropleth_map = alt.Chart(counties).mark_geoshape().encode(
    color=alt.Color('rate:Q', scale=alt.Scale(range=['snow', 'red'])),
    tooltip=[alt.Tooltip('id:O', title='County Code'),
             alt.Tooltip('rate:Q', title='Unemployment Rate(%)')]
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(source, 'id',['rate'])
).transform_calculate(
    rate='datum.rate * 100'  # Convert rate to percentage
).properties(
    width=1080,
    height=720,
    title='US Unemployment Rate by County'
).project(
    type='albersUsa'
)

choropleth_map

alt.Chart(...)

In [7]:
counties

UrlData({
  format: TopoDataFormat({
    feature: 'counties',
    type: 'topojson'
  }),
  url: 'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/us-10m.json'
})

## Read JSON file from Google Drive (Gmail Account)

In [ ]:
with open('/content/gdrive/MyDrive/W209_DataViz/duto_guerra_tweets.json') as f:
  file = f.read()
  data = json.loads(file)

# two hashtags appear together in a tweet
print(f"len(data) = {len(data)}\nhashtags: count_edges = {len(data[181]['entities']['hashtags'])}, {data[181]['entities']['hashtags']}\n")
data[181]

len(data) = 5872
hashtags: count_edges = 3, [{'text': 'rstudioconf', 'indices': ['59', '71']}, {'text': 'DataViz', 'indices': ['138', '146']}, {'text': 'DataScience', 'indices': ['147', '159']}]



{'retweeted': False,
 'source': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
 'entities': {'user_mentions': [{'name': 'RStudio',
    'screen_name': 'rstudio',
    'indices': ['18', '26'],
    'id_str': '235261861',
    'id': '235261861'}],
  'urls': [{'url': 'https://t.co/sN6CjOYIhH',
    'expanded_url': 'http://johnguerra.co/viz/influentials/RStudioConf2018/',
    'display_url': 'johnguerra.co/viz/influentia…',
    'indices': ['114', '137']}],
  'symbols': [],
  'media': [{'expanded_url': 'https://twitter.com/duto_guerra/status/959841361070567425/photo/1',
    'indices': ['160', '183'],
    'url': 'https://t.co/FzHtxIYfVP',
    'media_url': 'http://pbs.twimg.com/tweet_video_thumb/DVIKSrBWkAAJuEG.jpg',
    'id_str': '959840986485657600',
    'id': '959840986485657600',
    'media_url_https': 'https://pbs.twimg.com/tweet_video_thumb/DVIKSrBWkAAJuEG.jpg',
    'sizes': {'medium': {'w': '600', 'h': '558', 'resize': 'fit'},
     'thumb': {'w': '150', 'h': '150', 're

In [ ]:
lt = data[181]['entities']['hashtags']
print("A tweet that contains three hashtags (=>'rstudioconf','DataViz','DataScience'):\n\n",lt)

A tweet that contains three hashtags (=>'rstudioconf','DataViz','DataScience'):

 [{'text': 'rstudioconf', 'indices': ['59', '71']}, {'text': 'DataViz', 'indices': ['138', '146']}, {'text': 'DataScience', 'indices': ['147', '159']}]
